In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
     sys.path.insert(0, project_root)

import json
from dotenv import load_dotenv
load_dotenv()
from supabase import create_client

from src.llm.tools.ChunkRetriever import RetrievalService
from src.llm.OpenAIClient import OpenAIClient
from src.storage.SupabaseService import SupabaseService

# ========== Constants and Environment Variables =========
YOUR_APP_DOMAIN = "www.stackifier.com"
TEST_EMAIL = os.environ.get("TEST_EMAIL")
TEST_PASSWORD = os.environ.get("TEST_PASSWORD")
if not TEST_EMAIL or not TEST_PASSWORD:
    raise ValueError("TEST_EMAIL and TEST_PASSWORD must be set in your .env file.")

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_ANON_KEY")
if not supabase_url or not supabase_key:
    raise ValueError("SUPABASE_URL and SUPABASE_ANON_KEY must be set in your .env file.")

auth_client_local = create_client(supabase_url, supabase_key)

print(f"Attempting to sign in with email: {TEST_EMAIL}")
auth_response = auth_client_local.auth.sign_in_with_password(
    {"email": TEST_EMAIL, "password": TEST_PASSWORD}
)
if not auth_response or not auth_response.user:
    error_detail = auth_response.error.message if hasattr(auth_response, 'error') and auth_response.error else "Unknown authentication error"
    raise ConnectionError(f"Supabase authentication failed: {error_detail}. Check credentials and Supabase Auth settings.")
authenticated_user_id_str_local = str(auth_response.user.id)
print(f"Authentication successful. User ID: {authenticated_user_id_str_local}")

Attempting to sign in with email: user@example.com
Authentication successful. User ID: 372ec112-6fd7-46a7-bab4-abf7623fb05b


In [ ]:
retrieval_service_main = RetrievalService(
        openai_client = OpenAIClient(),
        supabase_service = SupabaseService(supabase_client=auth_client_local)
    )

def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    for func in functions:
        # 1. Autopick tool
        if func.__name__ == function_name:
            return func(**function_args)
        
        # 2. retrieve_financial_chunks tool
        if "retrieve_financial_chunks" == function_name:
            function_result_json = retrieval_service_main.retrieve_chunks(
                    user_id=authenticated_user_id_str_local, # Pass user_id explicitly
                    **function_args
                )
            return function_result_json
        
        # 3. ERROR: Tool not found
        if func.__name__ != function_name:
            return f"Function {function_name} not found in the provided functions."

In [ ]:
import os
from google import genai
from google.genai import types
from src.llm.tools.PythonCalculatorTool import PythonCalculationTool
from src.helper.llm_helper_chat import create_final_answer_instructions

# Define the function declaration for the model
python_calculator_tool_local = PythonCalculationTool()
python_calculator_decleration = python_calculator_tool_local.get_tool_declaration_data()

# Define the function to execute the Python code
execute_python_calculations = python_calculator_tool_local.execute_python_calculations
all_tools_functions = [
    execute_python_calculations,
    
]

# Define the function declaration for the model
all_tools_decleration = [
    types.Tool(function_declarations=[python_calculator_decleration]),
    types.Tool(function_declarations=[RetrievalService.get_tool_declaration()])
]

config = {
    "tools": all_tools_decleration,
    "automatic_function_calling": {"disable": True},
    "tool_config": {"function_calling_config": {"mode": "auto"}},
}

# Configure the client
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
GEMINI_MODEL = "gemini-2.0-flash"
chat = client.chats.create(model=GEMINI_MODEL, config=config)
# prompt = "Show me annual report of tesla in 2021?" # "sum first 50 prime numbers, then divide by 0.04215215"
prompt = "Whats the Gross Carrying Amount for Total intangible assets for tesla in 2021? then divide the value by 0.04215215"

CONV_HISTORY = []
CONV_HISTORY.append(types.Content(role="user", parts=[types.Part(text=prompt)]))

turn_count = 0
max_turns = 5
while turn_count < max_turns:
    turn_count += 1
    print(f"\n\n---------------------Iteration count Start : {turn_count}---------------------\n\n")
    print(f"Conversation history before sending to model: {json.dumps([c.to_json_dict() for c in CONV_HISTORY], indent=4)}")

    response = client.models.generate_content(
        model=GEMINI_MODEL,
        contents=CONV_HISTORY,
        config = config,
    )
    content = response.candidates[0].content
    
    result = None
    
    if content.parts[0].function_call:
        CONV_HISTORY.append(content)
        result = str(call_function(content.parts[0].function_call, all_tools_functions))
        function_response_part = types.Part.from_function_response(name=content.parts[0].function_call.name,response={"output": result})
        CONV_HISTORY.append(types.Content(role="user", parts=[function_response_part]))
        final_instructions_text = create_final_answer_instructions(prompt, result, YOUR_APP_DOMAIN)
        CONV_HISTORY.append(types.Content(role="user", parts=[types.Part(text=final_instructions_text)]))
    else:
        if content.parts[0].text:
            CONV_HISTORY.append(content)
            final_agent_text_response = content.parts[0].text
            print(final_agent_text_response)
        else:
            print("(No text in content.parts[0], model might have finished or an issue occurred)")
            final_agent_text_response = "(Model provided no text in its final turn)"
            print(final_agent_text_response)
            break
    print(f"\n\n---------------------Iteration count End   : {turn_count}---------------------\n\n")

In [ ]:
from google import genai
from google.genai import types
import os

GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

client = genai.Client(api_key=GEMINI_API_KEY)
prompt = "What is the sum of the first 50 prime numbers?"
response = client.models.generate_content(
  model="gemini-2.5-flash-preview-05-20",
  contents=prompt,
  config=types.GenerateContentConfig(
    thinking_config=types.ThinkingConfig(
      include_thoughts=True
    )
  )
)

for part in response.candidates[0].content.parts:
  if not part.text:
    continue
  if part.thought:
    print("Thought summary:")
    print(part.text)
    print()
  else:
    print("Answer:")
    print(part.text)
    print()

In [ ]:
import json
print(f"Response with thought:\n{json.dumps([c.to_json_dict() for c in response.candidates], indent=4)}")


In [2]:

import sys
import os

from api.v1.dependencies import SUPABASE_KEY, SUPABASE_URL, Session
from src.llm.workflow.react_rag import run_react_rag
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
import asyncio
# Test run_react_rag function
session = Session(user_id='test_user', token=SUPABASE_KEY)
test_client = create_client(SUPABASE_URL, SUPABASE_KEY)
import nest_asyncio
nest_asyncio.apply()
async def main_test():
    print('[TEST] Starting run_react_rag test')
    async for chunk in run_react_rag(session, test_client, 'Hello RAG test', []):
        print(f'[TEST] Received chunk: {chunk}')
    print('[TEST] run_react_rag test completed')
asyncio.run(main_test())

[TEST] Starting run_react_rag test
[DEBUG] run_react_rag called with session.user_id=test_user, user_input=Hello RAG test, history_len=0
Initialized OpenAI client with model: text-embedding-3-small
SupabaseService initialized with provided client.
[DEBUG] message received: Hello
[TEST] Received chunk: Hello
[DEBUG] message received: ! How can I help you today?

[TEST] Received chunk: ! How can I help you today?

[DEBUG] run_react_rag completed streaming
[TEST] run_react_rag test completed
